In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

import random
import numpy as np
import torch
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 演習：血液細胞計数

顕微鏡画像の解析に深層学習を活用することで、作業の効率化が期待されています。例えば、基礎研究において、顕微鏡を使って細胞を種類ごとに分類し、その数を調べる作業が行われています。本節では、このような物体検出と個数カウントの実例として、顕微鏡で撮影した血液細胞の画像から赤血球、白血球、血小板を検出し、それぞれの個数を数える方法を学びます。

## 演習準備

### ライブラリ

本節で利用するライブラリを読み込みます。NumPy、Pnadas、Matplotlib、Pillow（PIL）などのライブライは、モデルの性能や推論結果などの可視化に利用します。scikit-learn（sklearn）、PyTorch（torch）、torchvision、torchmetrics は機械学習関連のライブラリであり、モデルの構築、検証や推論などに利用します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image

import torch
import torchvision
import torchmetrics

print(f'torch v{torch.__version__}; torchvision v{torchvision.__version__}')

ライブラリの読み込み時に、*ImportError* や *ModuleNotFoundError* が発生した場合、必要なライブラリがインストールされていない可能性があります。該当ライブラリをインストールしてください。

### データセット

本節では、BCCD (blood cell count and detection) データセットを利用します。このデータセットは顕微鏡を利用して撮影された血液細胞の画像で、主に赤血球（red blood cell; RBC）、白血球（white blood cell; WBC）、血小板（platelet）を含みます。BCCD データセットは MIT ライセンスのもとで GitHub [BCCD_Dataset](https://github.com/Shenggan/BCCD_Dataset/tree/master) で公開されており、著作権表示を行うことで自由に利用することができます。


```{figure} ../_static/bccd_dataset.jpg
---
name: bccd_dataset_example
---
BCCD データセットに含まれる顕微鏡画像のサンプル。
```

本節では、オリジナルの BCCD データセットを再整理して作成したデータセットを利用します。本節で利用するデータセットは訓練、検証、そしてテストの 3 つのサブセットからなり、それぞれのサブセットにおいて各カテゴリに 50 枚、20 枚、20 枚の画像が含まれています。

Jupyter Notebook では、次のコマンドを実行することで、データをダウンロードできます。

```bash
!wget https://medDL.biopapyrus.jp/datasets/bccd.zip
!unzip bccd.zip
```

### 前処理

物体検出の問題では、画像と一緒に、検出対象の物体を囲むバウンディングボックスの座標とラベルをモデルに与え、学習させる必要があります。バウンディングボックスの座標とラベルは、一般的に COCO フォーマット（`.json`）や Pascal VOC フォーマット（`.xml`）などで保存されます。しかし、これらのフォーマットのままでは PyTorch で直接扱えないため、PyTorch が利用できる形式に変換する必要があります。

In [ ]:
class CocoDataset(torchvision.datasets.CocoDetection):
    def __init__(self, root, annFile):
        super(CocoDataset, self).__init__(root, annFile)
    
    def __getitem__(self, idx):
        img, target = super(CocoDataset, self).__getitem__(idx)
        
        boxes = []
        labels = []
        for obj in target:
            bbox = obj['bbox']
            bbox = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
            boxes.append(bbox)
            labels.append(obj['category_id'])
        
        img = torchvision.transforms.functional.to_tensor(img)
        target = {
            'boxes': torch.as_tensor(boxes, dtype=torch.float32),
            'labels': torch.as_tensor(labels, dtype=torch.int64),
        }

        return img, target
    

なお、画像分類と同様に、畳み込みニューラルネットワークでは入力する画像のサイズを指定されたサイズに変更する必要があります。ただし、このサイズ変更は `CocoDetection` 内で行われるため、ここであらためて設定する必要はありません。

また、モデルを訓練する際には、画像の拡大縮小や平行移動、回転などのデータ拡張を行い、それに伴ってバウンディングボックスの座標も同じように再計算する必要があります。しかし、これらの処理を加えるとコードが複雑化し、全体の流れがわかりにくくなるため、本節ではデータ拡張の処理は省略しています。

### 計算デバイス

PyTorch が GPU を認識できる場合は GPU を利用し、認識できない場合は CPU を利用するように計算デバイスを設定します。

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## モデル構築

SSD（Single Shot MultiBox Detector）は、高性能な物体検出アーキテクチャであり、`torchvision.models` モジュールを通じて利用できます。しかし、このアーキテクチャは COCO データセット向けに設計されており、車や人などの 90 種類の一般的なオブジェクトを検出するための構造になっています。そのため、そのままでは血液細胞（赤血球、白血球、血小板）の検出には対応していません。

血液細胞の検出を行うには、このアーキテクチャの分類部分の出力数を、検出対象となる 3 カテゴリ（赤血球、白血球、血小板）に対応させる必要があります。この修正はモデルを呼び出すたびに同じ作業を繰り返す必要があり、手間がかかります。そこで、指定したクラス数に応じてアーキテクチャを生成し、必要に応じて修正を加えられるよう、一連の処理を関数として定義します。

In [ ]:
def ssd(num_classes, weights=None):
    model = torchvision.models.detection.ssd300_vgg16(weights='DEFAULT')
    
    in_channels = torchvision.models.detection._utils.retrieve_out_channels(model.backbone, (300, 300))
    num_anchors = model.anchor_generator.num_anchors_per_location()
    model.head.classification_head = torchvision.models.detection.ssd.SSDClassificationHead(
        in_channels=in_channels,
        num_anchors=num_anchors,
        num_classes=num_classes + 1,
    )
    model.transform.min_size = (300,)
    model.transform.max_size = 3000
    if weights is not None:
        model.load_state_dict(torch.load(weights))
    return model

model = ssd(num_classes=3)
model.to(device)

## モデル訓練

モデルが学習データを効率よく学習できるようにするため、学習アルゴリズム（`optimizer`）、学習率（`lr`）、および学習率を調整するスケジューラ（`lr_scheduler`）を設定します。

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

次に、訓練データと検証データを読み込み、モデルが入力できる形式に整えます。

In [ ]:
train_loader = torch.utils.data.DataLoader(
                    CocoDataset('bccd', 'bccd/train.bbox.json'),
                    batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
valid_loader = torch.utils.data.DataLoader(
                    CocoDataset('bccd', 'bccd/valid.bbox.json'),
                    batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

準備が整ったら、訓練を開始します。訓練プロセスでは、訓練と検証を交互に繰り返します。訓練では、訓練データを使ってモデルのパラメータを更新し、その際の損失（誤差）を記録します。検証では、検証データを使ってモデルの予測性能（mAP）を計算し、その結果を記録します。

In [ ]:
num_epochs = 10
metric_dict = []

for epoch in range(num_epochs):
    # training phase
    model.train()
    epoch_loss_dict = {}

    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        batch_loss_dict = model(images, targets)
        batch_tol_loss = 0
        for loss_type, loss_val in batch_loss_dict.items():
            batch_tol_loss += loss_val
            if loss_type in epoch_loss_dict:
                epoch_loss_dict[f'train_{loss_type}'] += loss_val.item()
            else:
                epoch_loss_dict[f'train_{loss_type}'] = loss_val.item()
                
        # update weights
        optimizer.zero_grad()
        batch_tol_loss.backward()
        optimizer.step()
    lr_scheduler.step()

    # validation phase
    model.eval()
    metric = torchmetrics.detection.mean_ap.MeanAveragePrecision()
    with torch.no_grad():
        for images, targets in valid_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            metric.update(model(images), targets)


    # record training loss
    epoch_loss_dict['train_loss_total'] = sum(epoch_loss_dict.values())
    metric_dict.append({k: v / len(train_loader) for k, v in epoch_loss_dict.items()})
    for k, v in metric.compute().items():
        if k != 'classes':
            metric_dict[-1][k] = v.item()
    metric_dict[-1]['epoch'] = epoch + 1

    print(metric_dict[-1])


訓練後、訓練中の損失および検証性能の変化を可視化し、訓練が正しく行われたかどうかを確認します。

In [ ]:
metric_dict = pd.DataFrame(metric_dict)

fig, ax = plt.subplots(1, 2)
ax[0].plot(metric_dict['epoch'], metric_dict['train_loss_total'], label='total')
ax[0].plot(metric_dict['epoch'], metric_dict['train_classification'], label='classification')
ax[0].plot(metric_dict['epoch'], metric_dict['train_bbox_regression'], label='bbox')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].set_title('Train')
ax[0].legend()
ax[1].plot(metric_dict['epoch'], metric_dict['map_50'])
ax[1].set_ylim(0, 1)
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('mAP (50%)')
ax[1].set_title('Validation')
plt.tight_layout()
fig.show()

In [ ]:
glue('valid_map', metric_dict['map_50'].iloc[5:].mean(), display=False)

訓練データに対する損失はエポック数の増加に伴って徐々に減少しており、一部で上下するものの、後半では収束しているように見えます。また、検証データに対する検出性能（mAP 50%）は、4 エポック目以降安定して {glue:text}`valid_map:.3f` 前後で推移しています。この結果から、さらなる性能改善を目指す場合には、データセットを増やしたりする必要があるかもしれません。ただし、本節では時間の関係上、ここで訓練を終了します。

次に、この手順を Faster R-CNN や YOLO など、他の深層ニューラルネットワークアーキテクチャに適用し、それぞれの検証性能を比較して、このデータセットに最適なアーキテクチャを選定するプロセスを考えます。ただし、本節では時間の都合上、他のアーキテクチャを構築せず、既に構築した SSD を最適なアーキテクチャと仮定して次のステップに進みます。

次のステップでは、訓練サブセットと検証サブセットを統合したデータセット（`trainvalid.bbox.json`）を用いて、最適と判断したアーキテクチャを最初から訓練します。

In [ ]:
# model
model = ssd(num_classes=3)
model.to(device)

# training parameters
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# training data
train_loader = torch.utils.data.DataLoader(
                    CocoDataset('bccd', 'bccd/trainvalid.bbox.json'),
                    batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# training
num_epochs = 5
metric_dict = []
for epoch in range(num_epochs):
    model.train()
    epoch_loss_dict = {}
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        batch_loss_dict = model(images, targets)
        batch_tol_loss = 0
        for loss_type, loss_val in batch_loss_dict.items():
            batch_tol_loss += loss_val
            if loss_type in epoch_loss_dict:
                epoch_loss_dict[f'train_{loss_type}'] += loss_val.item()
            else:
                epoch_loss_dict[f'train_{loss_type}'] = loss_val.item()
                
        # update weights
        optimizer.zero_grad()
        batch_tol_loss.backward()
        optimizer.step()
    lr_scheduler.step()

    # record training loss
    epoch_loss_dict['train_loss_total'] = sum(epoch_loss_dict.values())
    metric_dict.append({k: v / len(train_loader) for k, v in epoch_loss_dict.items()})
    metric_dict[-1]['epoch'] = epoch + 1

    print(metric_dict[-1])


訓練が完了したら、訓練済みモデルの重みをファイルに保存します。

In [ ]:
model.to('cpu')
torch.save(model.state_dict(), 'bccd.pth')

## モデル評価

最適なモデルが得られたら、次にテストデータを用いてモデルを詳細に評価します。

In [ ]:
test_loader = torch.utils.data.DataLoader(
                    CocoDataset('bccd', 'bccd/test.bbox.json'),
                    batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

model = ssd(num_classes=3, weights='bccd.pth')
model.to(device)
model.eval()

n_cells = [[], [], []]
n_pred_cells = [[], [], []]

metric = torchmetrics.detection.mean_ap.MeanAveragePrecision()
with torch.no_grad():
    for images, targets in test_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        outputs = model(images)

        for target, output in zip(targets, outputs):
            n_cells_ = [0, 0, 0]
            n_pred_cells_ = [0, 0, 0]
            for label in target['labels']:
                n_cells_[label.item() - 1] += 1
            for label, score in zip(output['labels'], output['scores']):
                if score > 0.5:
                    n_pred_cells_[label.item() - 1] += 1
            for i in range(3):
                n_cells[i].append(n_cells_[i])
                n_pred_cells[i].append(n_pred_cells_[i])
        metric.update(outputs, targets)

n_cells = pd.DataFrame(n_cells, index=['WBC', 'RBC', 'Platelets']).T
n_pred_cells = pd.DataFrame(n_pred_cells, index=['WBC', 'RBC', 'Platelets']).T

metrics = {}
for k, v in metric.compute().items():
    metrics[k] = v.cpu().detach().numpy().tolist()

In [ ]:
metrics

In [ ]:
glue('valid_map50', metrics['map_50'], display=False)

テストデータに対する検出性能を確認したところ、mAP（50%）が {glue:text}`valid_map50:.3f` 程度であり、まだ改善の余地があるかもしれません。時間の関係上、モデルのチューニングをここで終了します。次に、モデルが予測した細胞の数と、実際にアノテーションされた細胞の数を比較するために、画像上で結果を可視化します。

In [ ]:
fig, ax = plt.subplots()
for i in range(3):
    ax.scatter(n_cells.iloc[:, i], n_pred_cells.iloc[:, i], label=n_cells.columns[i], alpha=0.5)
ax.set_xlabel('Number of Cells')
ax.set_ylabel('Number of Predicted Cells')
ax.set_aspect('equal')
ax.legend()
fig.show()

可視化の結果、白血球と赤血球の予測数はアノテーションされた数と似た数となることを確認できます。しかし、血小板についてはほとんど予測できていないことがわかります。これをより正確に評価するには、RMSE（平方平均二乗誤差）などの指標を用いる必要がありますが、ここで省略します。

血小板の予測が困難だった理由の一つは、SSD が持つ特徴と考えられます。SSD は大きな物体を高速かつ高精度で検出するのに優れている一方で、血小板のような小さな物体の検出は得意ではありません。また、今回使用したデータセットでは、血小板のアノテーション数が非常に少なかったことも影響していると考えられます。その結果、SSD は血小板のような小さな物体の特徴を十分に学習できなかった可能性があります。このように、大きな物体と小さな物体が混在する画像で、特に小さな物体のアノテーション数が著しく少ない場合、SSD は必ずしも最適な選択ではないと考えられます。

## 推論

推論時にも、訓練時と同じように torchvision モジュールからアーキテクチャを呼び出し、出力層のクラス数を設定します。その後、`load_state_dict` メソッドを使って、訓練済みの重みファイルをモデルにロードします。これらの操作はすべて `ssd` 関数で定義されているので、その関数を利用します。

In [ ]:
model = ssd(num_classes=3, weights='bccd.pth')
model.to(device)
model.eval()

このモデルを利用して推論を行います。まず、1 枚の画像を指定し、PIL モジュールを用いて画像を開き、テンソル形式に変換した後、モデルに入力します。モデルは予測結果としてバウンディングボックスの座標（`bboxes`）、分類ラベル（`labels`）、および信頼スコア（`scores`）を出力します。ただし、信頼スコアが 0.5 未満のバウンディングボックスは採用せず、信頼スコアが高い結果のみを選択して利用します。

In [ ]:
def predict(image_path, threshold=0.5):
    image = PIL.Image.open(image_path).convert('RGB')
    input_tensor = torchvision.transforms.functional.to_tensor(image).unsqueeze(0).to(device)
        
    with torch.no_grad():
        predictions = model(input_tensor)[0]
        
    bboxes = predictions['boxes'][predictions['scores'] > threshold]
    labels = predictions['labels'][predictions['scores'] > threshold]
    scores = predictions['scores'][predictions['scores'] > threshold]
    bboxes = [_.cpu().detach().numpy().tolist() for _ in bboxes]
    labels = [int(_.cpu().detach().item()) for _ in labels]
    scores = [float(_.cpu().detach().item()) for _ in scores]
    
    return bboxes, labels, scores

bboxes, labels, scores = predict('bccd/images/BloodImage_00033.jpg')

for bbox, label, score in zip(bboxes, labels, scores):
    print([bbox, label, score])


検出されたオブジェクトのバウンディングボックスを入力画像に描画します。その後、PIL および matplotlib ライブラリを使用して、画像とその検出結果を可視化します。

In [ ]:
def viz(image_path):
    bboxes, labels, scores = predict(image_path)
    meta_info = [
        {}, # background
        {'class': 'WBC', 'col': '#0094cd'},
        {'class': 'RBC', 'col': '#d51317'},
        {'class': 'Platelets', 'col': '#007b3d'}
    ]
    
    image = PIL.Image.open(image_path).convert('RGB')
    draw = PIL.ImageDraw.Draw(image)
    for bbox, label, score in zip(bboxes, labels, scores):
        x1, y1, x2, y2 = bbox
        draw.rectangle(((x1, y1), (x2, y2)), outline=meta_info[label]['col'], width=3)
        draw.text((x1, y1 - 10), f'{meta_info[label]["class"]} ({score:.2f})', fill=meta_info[label]['col'])
        
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(image)
    ax.axis('off')
    fig.show()


In [ ]:
viz('bccd/images/BloodImage_00033.jpg')